In [ ]:
! pip install jsonlines
! pip install fast_ml --quiet
! pip install transformers
! pip install nltk
! python -m nltk.downloader all
! pip install unidecode
! pip install textblob
from unidecode import unidecode
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import tensorflow as tf
import transformers
from textblob import TextBlob
import os
import json
import pandas as pd

In [2]:
# Import tagged files
import jsonlines
list_file = jsonlines.open('/content/drive/MyDrive/Dissertation/admin.jsonl')
documents=[]
for i in list_file:
  documents.append(i)
text = documents[0].get("text")
entities = documents[0].get("entities")

In [3]:
# functions to index the text
def index(document):
  training = pd.DataFrame(columns=['label','Text'])
  for i in range(0,len(document)):
    text = documents[i].get("text")
    entities = documents[i].get("entities")
    for z in range(0,len(entities)):
      start_offset = entities[z].get('start_offset')
      end_offset = entities[z].get('end_offset')
      label = entities[z].get('label')
      text_ext = text[start_offset:end_offset]
      list_row = {'label': label, 'Text': text_ext}
      training = training.append(list_row,ignore_index=True)
      z+=1
    i+=1
  return training
result = index(documents)

<ipython-input-3-a8e4cc914aca>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  training = training.append(list_row,ignore_index=True)
<ipython-input-3-a8e4cc914aca>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  training = training.append(list_row,ignore_index=True)
<ipython-input-3-a8e4cc914aca>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  training = training.append(list_row,ignore_index=True)
<ipython-input-3-a8e4cc914aca>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  training = training.append(list_row,ignore_index=True)
<ipython-input-3-a8e4cc914aca>:14: FutureWarning: The frame.append method is deprecated and will be 

In [5]:
import os
import json
# assign directory
directory = '/content/drive/MyDrive/Dissertation/text_file'

# iterate over files in
# that directory
directory_list = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        directory_list.append(f)

# Extra Json Files with labelling
# Import and Remove empty lablled text
def loading(directory):
  df_extra = pd.DataFrame(columns = ['label','Text'])
  for i in range(0,len(directory)):
    path = directory[i]
    f= open(path)
    data = json.load(f)
    data = list(filter(lambda x: x['label'] != [], data))
    for z in range(0,len(data)):
      text = data[z]['text']
      label = data[z]['label'][0]
      list_row = {'label':label,'Text':text}
      df_extra = df_extra.append(list_row, ignore_index=True)
      z+=1
    i+=1
  return df_extra
# Load into dataframe
df_extra = loading(directory_list)

<ipython-input-5-769254c20683>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_extra = df_extra.append(list_row, ignore_index=True)
<ipython-input-5-769254c20683>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_extra = df_extra.append(list_row, ignore_index=True)
<ipython-input-5-769254c20683>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_extra = df_extra.append(list_row, ignore_index=True)
<ipython-input-5-769254c20683>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_extra = df_extra.append(list_row, ignore_index=True)
<ipython-input-5-769254c20683>:28: FutureWarning: The frame.append method is deprecated and will

In [6]:
# Append two dataframe together
df_total = pd.concat([result,df_extra])

In [7]:
# Preprocess the dataset for word embedding
def pre_process(text):
    # convert input corpus to lower case.
    text = text.lower()
    # collecting a list of stop words from nltk and punctuation form
    # string class and create single array.
    stopset = stopwords.words('english') + list(string.punctuation)
    # remove stop words and punctuations from string.
    # word_tokenize is used to tokenize the input corpus in word tokens.
    text = " ".join([i for i in word_tokenize(text) if i not in stopset])
    # remove non-ascii characters
    text = unidecode(text)
    return text

# Lemmatization and spell check
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()

# Word Tokenisationb & Lemmatization
def word_token(sentence):
  words = word_tokenize(sentence)
  for w in range(0,len(words)):
    words[w]=lemmatizer.lemmatize(words[w])
    w+=1
  words = ' '.join(words)
  return words

# Spelling correction
def spelling(sentence):
  b = TextBlob(sentence).correct()
  b= str(b)
  return b

# Combination of preprocess spelling and lemetization funcitons
def lemmatization(dataset):
  processed_text = []
  for i in range(0,len(dataset)):
    text = dataset.iloc[i][1]
    text_new = pre_process(text)
    word_orig = word_token(text_new)
    word_orig_correct = spelling(word_orig)
    processed_text.append(word_orig_correct)
    i+=1
  dataset['processed_text'] = processed_text
  return dataset
# Apply to result
lemmatization(df_total)
df_total = df_total.drop('Text',axis=1)

In [8]:
# Remove unnecessary columns
remove_columns = ['servce_charge_cap','Intererst_3%_above','Alteration_erection','Alienation_share_occupation',
                  'Alienation_withholding_consent',
       'Alienation_condition_landlord_assign',
       'Alienation_undertenant_covenant', 'Alienation_underlease_term',
       'Alienation_tenant_convenant_novary',
       'Alienation_further_underlease_prohibit', 'Rights_tenant',
       'Rights_landlord', 'Rent_Review_disputes', 'Insurance_risks',
       'Insurance_insurance_payable_landlord', 'Insurance_determine',
       'Uninsured_landlord_reinstate', 'Uninsured_3_terminate','Demise_structure', 'Intererst_payable_due',
       'Intererst_grace_14', 'Outgoings', 'Alienation_landlord_AGA',
       'VAT_payable_prevailing', 'Forfeiture_21_day',
       'Forfeiture_tenantob', 'Forfeiture_creditors',
       'Forfeiture_liquidation','Repair_tenant_breach', 'Repair_tenant_cost_dil',
       'Alienation_assign_charge_prohibit', 'Alienation_charging_whole']
for i in remove_columns:
  df_total = df_total[df_total['label']!= i]

In [9]:
# Making the labelling consistent
re = ['Premises','Premise'],['Date_of_lease','DOL'],['Use_change','Permitted_use'],['Term_date','Term'],['servce_charge_tenant','service_proportion'],['Rent_payment_quarterly','Rent_payable_quarterly'],['Alteration_nonsturctural','Alteration_non_structural'],['Alienation_underletting_part','Underlet_part'],['Alienation_underletting_whole','Underlet_whole'],['Alienation_assign_whole_permit','Assignment_whole'],['Repair_tenant_premise_decorate','Repair_obligation'],['Alteration_decorate','Repair_decoration'],['Yielding_up','Yield_up'],['Insurance_landlord_reinstatement','Insurance_full_reinstatment'],['Rent_commence_date','Rent_Commencement_date'],['Term_tenant_determine','Tenant_determine'],['Rent_review_date','Remaining_Review_date'],['Rent_payment_deduction','Rent_deduction'],['Term_1954','Act_1954'],['Term_landlord_determine','Landlord_determine'],['Rent_Review_openmarket','Rent_Review_method'],['current_tenant','Current_Tenant'],['guarantor','Guarantor'],['service_charge','service_proportion']
for i in range(0,len(re)):
  df_total['label']= df_total['label'].replace(re[i][0],re[i][1])
  i+=1

In [10]:
# Map Label to columns
df_total['label_cat'] = df_total['label']
df_total['label_cat'] = pd.factorize(df_total['label_cat'])[0]

In [12]:
# Load data
df_total = pd.read_parquet('/content/drive/MyDrive/Dissertation/df_total.parquet')
# One-shot sample
df_label_unique_sample = df_total.groupby('label_cat', group_keys=False).apply(lambda df: df.sample(1))

# Create label mapping
label = list(df_label_unique_sample['label'])
id = list(df_label_unique_sample['label_cat'])
label_to_id = dict(zip(label,id))
id_to_label = dict(zip(id,label))

In [ ]:
df_total.to_parquet('/content/drive/MyDrive/Dissertation/df_total.parquet')

In [ ]:
# Extract columns to match the dataset requirement for semantic analysis, split into training, test, validation dataset
# train, test, validation split
from fast_ml.model_development import train_valid_test_split
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df_total,
                                                                            target = 'label_cat',
                                                                            train_size=0.7,
                                                                            valid_size=0.2,
                                                                            test_size=0.1)
# reset index
for data in [X_train, y_train, X_valid, y_valid, X_test, y_test]:
    data.reset_index(drop=True, inplace=True)